In [26]:
import cv2
import numpy as  np
import tensorflow as tf
import tensorflow.keras
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten,concatenate,AveragePooling2D,Rescaling,Conv2D,MaxPooling2D,BatchNormalization,LeakyReLU,GlobalAvgPool2D

In [27]:
def crop_minAreaRect(rect, box, gray):
    mult = 1 
    W = rect[1][0]
    H = rect[1][1]

    Xs = [i[0] for i in box]
    Ys = [i[1] for i in box]
    x1 = min(Xs)
    x2 = max(Xs)
    y1 = min(Ys)
    y2 = max(Ys)

    rotated = False
    angle = rect[2]

    if angle < -45:
        angle+=90
        rotated = True

    center = (int((x1+x2)/2), int((y1+y2)/2))
    size = (int(mult*(x2-x1)),int(mult*(y2-y1)))

    M = cv2.getRotationMatrix2D((size[0]/2, size[1]/2), angle, 1.0)

    cropped = cv2.getRectSubPix(gray, size, center)    
    cropped = cv2.warpAffine(cropped, M, size)

    croppedW = W if not rotated else H 
    croppedH = H if not rotated else W

    croppedRotated = cv2.getRectSubPix(cropped, (int(croppedW*mult), int(croppedH*mult)), (size[0]/2, size[1]/2))
    
    return croppedRotated

In [28]:
def conv_block(x, filters, kernel_size, name=None):
  x = Conv2D(filters=filters, kernel_size=kernel_size, padding='same', name=name, use_bias=False)(x)
  x = BatchNormalization(name=name if name is None else '%s_BatchNorm' %name)(x)
  x = LeakyReLU(alpha=0.1, name=name if name is None else '%s_LeakyRelu' %name)(x)
  return x

In [29]:
input_shape = (64, 64, 3)


inputs = tf.keras.Input(shape=input_shape)

x = conv_block(inputs, filters=8, kernel_size=(3,3), name='conv1_3x3')
x = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same', name='maxpool1_2x2/2')(x)
x = conv_block(x, filters=16, kernel_size=(3,3), name='conv2_3x3')
x = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same', name='maxpool2_2x2/2')(x)

x = conv_block(x, filters=32, kernel_size=(3,3), name='conv3_3x3')
x = conv_block(x, filters=16, kernel_size=(1,1), name='conv4_1x1')
x = conv_block(x, filters=32, kernel_size=(3,3), name='conv5_3x3')
x = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same', name='maxpool3_2x2/2')(x)

x = conv_block(x, filters=64, kernel_size=(3,3), name='conv6_3x3')
x = conv_block(x, filters=32, kernel_size=(1,1), name='conv7_1x1')
x = conv_block(x, filters=64, kernel_size=(3,3), name='conv8_3x3')
x = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same', name='maxpool4_2x2/2')(x)

x = conv_block(x, filters=128, kernel_size=(3,3), name='conv9_3x3')
x = conv_block(x, filters=64, kernel_size=(1,1), name='conv10_1x1')
x = conv_block(x, filters=128, kernel_size=(3,3), name='conv11_3x3')
x = conv_block(x, filters=64, kernel_size=(1,1), name='conv12_1x1')
x = conv_block(x, filters=128, kernel_size=(3,3), name='conv13_3x3')
x = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same', name='maxpool5_2x2/2')(x)

x = conv_block(x, filters=256, kernel_size=(3,3), name='conv14_3x3')
x = conv_block(x, filters=128, kernel_size=(1,1), name='conv15_1x1')
x = conv_block(x, filters=256, kernel_size=(3,3), name='conv16_3x3')
x = conv_block(x, filters=128, kernel_size=(1,1), name='conv17_1x1')
x = conv_block(x, filters=256, kernel_size=(3,3), name='conv18_3x3')

x = conv_block(x, filters=250, kernel_size=(1,1), name='conv19_1x1')
x = GlobalAvgPool2D()(x)
x = Flatten()(x)
outputs = Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 conv1_3x3 (Conv2D)          (None, 64, 64, 8)         216       
                                                                 
 conv1_3x3_BatchNorm (BatchN  (None, 64, 64, 8)        32        
 ormalization)                                                   
                                                                 
 conv1_3x3_LeakyRelu (LeakyR  (None, 64, 64, 8)        0         
 eLU)                                                            
                                                                 
 maxpool1_2x2/2 (MaxPooling2  (None, 32, 32, 8)        0         
 D)                                                              
                                                           

In [30]:
model = tf.keras.models.load_model("model_darknet19.h5")

In [31]:
from keras.preprocessing.image import img_to_array


In [32]:
img_crop=cv2.imread("2.jpg",1)
img_crop = cv2.resize(img_crop, (64,64), interpolation = cv2.INTER_AREA)
img_crop = img_to_array(img_crop)
img_crop = img_crop.reshape((1, img_crop.shape[0], img_crop.shape[1], img_crop.shape[2]))
Y_box_pred = model.predict(img_crop)
print(Y_box_pred)

[[0.9999975]]


In [33]:
video_path='test2.mp4'
cap=cv2.VideoCapture(video_path)
backSub=cv2.createBackgroundSubtractorMOG2()
if(cap.isOpened()==False):
    print("Khong mo duoc video")
while (cap.isOpened()):
    ret, frame=cap.read()
    if ret==True:
        #Thuat toan tru nen
        fgMask=backSub.apply(frame)
        fgMask=cv2.cvtColor(fgMask, 0)
        
        #Dung bo loc ma tran 5*5
        kernel = np.ones((7,7), np.uint8)
        #Lay gia tri pixel nho nhat cua kernel lam manh net
        fgMask=cv2.erode(fgMask,kernel,iterations=1)
        #Lay gia tri pixel lon nhat cua kernel lam lien net
        fgMask=cv2.dilate(fgMask,kernel,iterations=1)
        
        #tao them su lien net bang ham open
        fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_CLOSE, kernel,iterations=1)
        #lay nguong 120
        _,fgMask = cv2.threshold(fgMask,120,255,cv2.THRESH_BINARY)
        #phat hien canh bang candy
        fgMask=cv2.Canny(fgMask, 20, 200)
        contours,_ = cv2.findContours(fgMask,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for i in range(len(contours)):
            rect = cv2.minAreaRect(contours[i])
            if (rect[1][0]*rect[1][1]) < 1000:
                continue
            
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            
            img_crop = crop_minAreaRect(rect, box, frame)       
            img_crop = cv2.resize(img_crop, (64,64), interpolation = cv2.INTER_AREA)
            img_crop = img_to_array(img_crop)
            img_crop = img_crop.reshape((1, img_crop.shape[0], img_crop.shape[1], img_crop.shape[2]))
            cv2.drawContours(frame,[box],0,(0,255,0),2)
        
        
            Y_box_pred = model.predict(img_crop)
            if Y_box_pred <= 0.5:
                cv2.putText(frame, "Person", (box[2][0], box[2][1]),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            if Y_box_pred > 0.5:
                cv2.putText(frame, "Non", (box[2][0], box[2][1]),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

        cv2.imshow('Frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
        
cv2.destroyAllWindows()    
            